In [18]:
from pyspark.sql import SparkSession
import os
from datetime import timedelta,datetime
from pyspark.sql.functions import *
import time

In [19]:
spark = SparkSession \
    .builder \
        .config("spark.jars", "/mnt/postgres_jar/postgresql-42.5.0.jar") \
        .getOrCreate()    

In [20]:
base_path = '/mnt/data/parquet'
override_config_key = 'spark.override.date'
current_date = datetime.now().strftime("%Y-%m-%d")

In [21]:
try: 
    conf_value = spark.conf.get(override_config_key) 
    paths_to_read = '{}/date={}'.format(base_path,conf_value)
except:
    print("Parameter is not set")
    paths_to_read = '{}/date={}'.format(base_path,current_date)

Parameter is not set


In [22]:
df = spark.read.option("basePath",base_path) \
    .parquet(paths_to_read)

In [23]:
aggregated_dataframe = df.groupBy("source_name","date") \
    .agg(count(lit(1)).alias("sum_articles") \
        ,max("timestamp").alias("last_published_date")
        )

In [25]:
aggregated_dataframe.write.format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:7432/airflow") \
    .option("dbtable", "aggregated_articles") \
    .option("driver", "org.postgresql.Driver") \
    .option("user", "airflow") \
    .option("password", "airflow") \
    .mode("append") \
    .save()